In [1]:
import pandas as pd
import datetime as dt
import numpy as np

In [2]:
data = pd.ExcelFile("data.xlsx")

In [3]:
swipes = pd.read_excel(data, 'fob_swipes')
swipes = swipes[['id', 'date_time', "time"]]
swipes['etype'] = 'S'
swipes['time'] = pd.to_datetime(swipes.date_time)

sent_emails = pd.read_excel(data, 'sent_items')
sent_emails = sent_emails[['id', 'date_time']]
sent_emails['etype'] = 'E'
sent_emails['time'] = pd.to_datetime(sent_emails.date_time)

format = '%H:%M'

# sent_emails['time'] = pd.Series([val.time() for val in sent_emails['time']])

sent_emails.head()
sent_emails.dtypes


id                    int64
date_time    datetime64[ns]
etype                object
time         datetime64[ns]
dtype: object

In [4]:
swipes.head()
swipes.dtypes

id                    int64
date_time    datetime64[ns]
time         datetime64[ns]
etype                object
dtype: object

In [5]:
frames = [swipes, sent_emails]
events = pd.concat(frames, ignore_index=True)

/Users/vineet/Documents/natalia-case/env/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [6]:
events[2725:2736]

,date_time,etype,id,time
2725,2018-04-20 10:13:00,S,3725,2018-04-20 10:13:00
2726,2018-04-20 12:30:00,S,3726,2018-04-20 12:30:00
2727,2018-04-20 14:07:00,S,3727,2018-04-20 14:07:00
2728,2018-04-20 16:54:00,S,3728,2018-04-20 16:54:00
2729,2018-04-20 16:59:00,S,3729,2018-04-20 16:59:00
2730,2014-05-07 15:57:11,E,5000,2014-05-07 15:57:11
2731,2014-05-07 18:37:13,E,5001,2014-05-07 18:37:13
2732,2014-05-07 18:38:48,E,5002,2014-05-07 18:38:48
2733,2014-05-09 18:08:36,E,5003,2014-05-09 18:08:36
2734,2014-05-12 16:46:09,E,5004,2014-05-12 16:46:09


In [7]:
# for every day hours worked, day ends at last swipw between midnight and 8am
# group by weeks

events.index = events["date_time"]
#result.index

In [8]:
events.dtypes

date_time    datetime64[ns]
etype                object
id                    int64
time         datetime64[ns]
dtype: object

In [9]:
test_date = "2017-11-16"

In [10]:
x = events[test_date].date_time.max()
y = events[test_date].date_time.min()

x

Timestamp('2017-11-16 15:53:00')

In [11]:
y

Timestamp('2017-11-16 01:15:02')

In [12]:
print(x - y) 

0 days 14:37:58


In [13]:
#events[test_date].sort_values(by='date_time',ascending=True)
events.head()

,date_time,etype,id,time
date_time,,,,
2015-09-21 16:05:00,2015-09-21 16:05:00,S,1000,2015-09-21 16:05:00
2015-09-21 16:50:00,2015-09-21 16:50:00,S,1001,2015-09-21 16:50:00
2015-09-22 10:39:00,2015-09-22 10:39:00,S,1002,2015-09-22 10:39:00
2015-09-22 12:51:00,2015-09-22 12:51:00,S,1003,2015-09-22 12:51:00
2015-09-22 15:26:00,2015-09-22 15:26:00,S,1004,2015-09-22 15:26:00


In [14]:
events['date'] = [val.date() for val in pd.to_datetime(events['date_time'])]
events['hour'] = [val.time() for val in pd.to_datetime(events['date_time'])]

events.head()

# grouped_events = events.groupby("date")
# grouped_events
# grouped_events['time'].max() - grouped_events['time'].min()

,date_time,etype,id,time,date,hour
date_time,,,,,,
2015-09-21 16:05:00,2015-09-21 16:05:00,S,1000,2015-09-21 16:05:00,2015-09-21,16:05:00
2015-09-21 16:50:00,2015-09-21 16:50:00,S,1001,2015-09-21 16:50:00,2015-09-21,16:50:00
2015-09-22 10:39:00,2015-09-22 10:39:00,S,1002,2015-09-22 10:39:00,2015-09-22,10:39:00
2015-09-22 12:51:00,2015-09-22 12:51:00,S,1003,2015-09-22 12:51:00,2015-09-22,12:51:00
2015-09-22 15:26:00,2015-09-22 15:26:00,S,1004,2015-09-22 15:26:00,2015-09-22,15:26:00


In [15]:
midnight = dt.datetime.strptime("00:00:00", '%H:%M:%S').time()
sevenAM = dt.datetime.strptime("07:00:00", '%H:%M:%S').time()

def convert_night_dates(row):
    if(row['hour'] <= sevenAM):
        row['date'] = (row['date'] - dt.timedelta(1))

    return row



# def convert_night_dates(row):
#     if(row['hour'] <= sevenAM):
#         print(row['hour'])
#         num_hours = dt.timedelta(midnight + row['hour'].time)
#         print(num_hours)
#     return 0

corrected_events = events.apply(convert_night_dates, axis=1)

overflow_max = corrected_events[events.hour <= sevenAM].groupby(by="date").max()
overflow_max.head()

,date_time,etype,id,time,hour
date,,,,,
2014-11-11,2014-11-12 01:40:44,E,5088,2014-11-12 01:40:44,01:40:44
2015-04-06,2015-04-07 02:09:38,E,5322,2015-04-07 02:09:38,02:09:38
2015-04-09,2015-04-10 03:04:37,E,5375,2015-04-10 03:04:37,03:04:37
2015-04-14,2015-04-15 00:57:12,E,5409,2015-04-15 00:57:12,00:57:12
2015-09-30,2015-10-01 00:53:26,E,5889,2015-10-01 00:53:26,00:53:26


In [16]:
corrected_grouped_events = events.groupby("date")

def minimum(group):
    midnight = dt.datetime.strptime("00:00:00", '%H:%M:%S').time()
    sevenAM = dt.datetime.strptime("07:00:00", '%H:%M:%S').time()
    
    greaterThanSevenAM = group[group.hour >= sevenAM]
    return greaterThanSevenAM.min()

def maximum(group):
    midnight = dt.datetime.strptime("00:00:00", '%H:%M:%S').time()
    sevenAM = dt.datetime.strptime("07:00:00", '%H:%M:%S').time()
    
    lessThanSevenAM = group[group.hour < sevenAM]
    if(lessThanSevenAM.size > 0):
        return lessThanSevenAM.max()
    
    return group.max()

def compute_hours_worked(group):
    _max = maximum(group)
    _min = minimum(group)
    diff = _max.date_time - _min.date_time
    date = _max.date
    
    row = pd.DataFrame()
    row.set_index("date")
    
#     print(row)
    
    pass
    
#     new_row = pd.DataFrame()
#     print()


    
    

first_event_times = corrected_grouped_events.agg(minimum)

last_event_times = corrected_grouped_events.agg(maximum)

# hours_worked = corrected_grouped_events.agg(compute_hours_worked)

first_event_times.date_time - last_event_times.date_time

# hours_worked

TypeError: ufunc subtract cannot use operands with types dtype('O') and dtype('<M8[ns]')

In [ ]:
print("events\n")
print(corrected_events.head())
print("first event time\n")
print(first_event_times.head())
print("last event time\n")
print(last_event_times.head())
print("hours worked\n")
print(hours_worked.head())

In [ ]:
# def agg(group):
#     group.join(other=first_event_time, on="date", lsuffix="_caller", rsuffix="other")
    
#     return group

# # corrected_grouped_events.apply(agg)

# events['date']

In [ ]:
# dates = pd.date_range('2015-10-02', periods=100, freq='D')
dates = events['date']

In [ ]:
labels = ["first_event_time"]
result = pd.DataFrame(columns=labels)

first_times = first_event_times
last_times = last_event_times
# hours = hours_worked.to_frame()

firstAndLast = first_times.join(other=last_times, how="inner", lsuffix="First Event Time", rsuffix="Last Event Time")
firstAndLast.join(other=hours, how="inner", lsuffix="f", rsuffix="hours")

# hours_worked
firstAndLast

In [ ]:
labels = ["first_event_time"]
result = pd.DataFrame(index=dates, columns=labels)

first_times = first_event_time.to_frame()

first_times.time[0]

# for date in dates:
#     data = []
    
#     f = first_times[first_times['time'] == date]
#     print(f)
    
#     row = pd.DataFrame([f] , index=[date], columns=labels)
#     result = result.append(row)

# result

# dates
# result.append()
# result